In [42]:
# 3. Import libraries and modules
import numpy as np
np.random.seed(123)  # for reproducibility
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist

pieceMap = {
  '1': [0,0,0,0,0,0,0,0,0,0,0,0],
  'p': [0,0,0,0,0,0,0,0,0,0,0,1],
  'r': [0,0,0,0,0,0,0,0,0,0,1,0],
  'n': [0,0,0,0,0,0,0,0,0,1,0,0],
  'b': [0,0,0,0,0,0,0,0,1,0,0,0],
  'q': [0,0,0,0,0,0,0,1,0,0,0,0],
  'k': [0,0,0,0,0,0,1,0,0,0,0,0],
  'P': [0,0,0,0,0,1,0,0,0,0,0,0],
  'R': [0,0,0,0,1,0,0,0,0,0,0,0],
  'N': [0,0,0,1,0,0,0,0,0,0,0,1],
  'B': [0,0,1,0,0,0,0,0,0,0,0,1],
  'Q': [0,1,0,0,0,0,0,0,0,0,0,1],
  'K': [1,0,0,0,0,0,0,0,0,0,0,1]
}

X_vals = []
Y_vals = []

i, nRows = 0, 1000000
input_neurons = 773
#data_shape = (nRows, input_neurons)
castlingAssert = 0
with open('allData.shuffled.data','r') as data:
    boardLine = data.readline()
    
    while boardLine and i < nRows:
        boardData = boardLine.split(':')
        boardR = boardData[0].split('/')
        board = boardR[0:8]
        
        board[7] = board[7][0:8]
        
        score = float(boardData[1])
        
        
                
        # Append 12x 0-1 neurons for each of 8 positions in each line
        nBoard = []
        for line in board:
            for piece in line:
                nBoard.extend(pieceMap[piece])
        
        # Append 0-1 neuron for current player
        who = boardR[7][9]
        assert who == 'w' or who == 'b', 'line {0}: expected position 9 to be current player'.format(i)
        nBoard.append(0 if who == 'w' else 1)
        
        # Append 4x 0-1 neurons for castling
        castlingRaw = boardR[7][11:15]
        
        #print(i, castlingRaw)
        
        castling = list(filter(lambda c: c in ['-', 'K', 'k', 'Q','q'], castlingRaw))
        #print(i, castling)
        
        assert len(castling) > 0, 'line {0}: expected castling {1} to be some characters'.format(i, castling)
        
        if len(castling) < 4: castlingAssert += 1
        #assert all(c in ['-', 'K', 'k', 'Q','q', ' '] for c in castling), 'line {0}: expected {1} to be castling rights'.format(i, castling)
        nBoard.append(1 if 'k' in castling else 0)
        nBoard.append(1 if 'q' in castling else 0)
        nBoard.append(1 if 'K' in castling else 0)
        nBoard.append(1 if 'Q' in castling else 0)
        
        assert len(nBoard) == input_neurons, 'line {0}: expected {2} neurons per board, got {1}'.format(i, len(nBoard), input_neurons)
        
        X_vals.append(nBoard)
        #Y_vals.append(1 if float(boardData[2]) > 0 else 0)
#         print(boardData)
        Y_vals.append(float(boardData[1]))
        boardLine = data.readline()
        i += 1
    X_vals = np.array(X_vals)
    Y_vals = np.array(Y_vals)
    print('wat',X_vals.shape)
    print('Y', Y_vals[4:10])
    #Xraw = data.split('\n').map(lambda r: r.split('/'))
    #print(Xraw.shape)
    
# sanity check for if input makes sense ... since a small number of lines have 2 instead of 4 characters for castling rights
#assert castlingAssert / nRows < 0.2, 'expected most of the rows to have 4 character castling rights {0}'.format(castlingAssert / nRows)

# 4. Load pre-shuffled MNIST data into train and test sets

split = nRows // 4
(X_train, y_train) = X_vals[0:3*split], Y_vals[0: 3*split].reshape(3*split,1)
(X_test, y_test) = X_vals[3*split :], Y_vals[3*split :].reshape(split,1)
#(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)

print(X_train[0])

#n = 6000
#X_train = X_train[0:n]
#y_train = y_train[0:n]
#X_test = X_test[0:n]
#y_test = y_test[0:n]

# 5. Preprocess input data
X_train = X_train.reshape(X_train.shape[0], input_neurons)
X_test = X_test.reshape(X_test.shape[0], input_neurons)
#X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
#X_train /= 12
#X_test /= 12

print(y_train)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
 
print("done")

wat (1000000, 773)
Y [-0.02  0.1  -0.42  7.38  0.27  0.86]
(750000, 773)
[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [43]:
# 6. Preprocess class labels
print(y_train[0:20])
print(max(y_train))
print(min(y_train))

#Y_train = np_utils.normalize(y_train)
#Y_test = np_utils.normalize(y_test)
print(Y_train[0:20])
print(max(Y_train))
print(min(Y_train))

Y_train = y_train
Y_test = y_test
#Y_train = np_utils.to_categorical(y_train, 2)
#Y_test = np_utils.to_categorical(y_test, 2)
 
# 7. Define model architecture
model = Sequential()
 
# model.add(Convolution2D(32, (3, 3), activation='relu', input_shape=(1,28,28)))
model.add(Dense(600, activation='relu', input_shape=[input_neurons]))
model.add(Dense(400, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(100, activation='relu'))

model.add(Dropout(0.25))

#model.add(Flatten())
model.add(Dense(1, activation='linear'))
#model.compile(loss='categorical_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])

print('compile')

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

print('summary')

model.summary()

# 9. Fit model on training data
print(X_train.shape, Y_train.shape)
model.fit(X_train, Y_train, 
          batch_size=100, epochs=10, verbose=1)

# 10. Evaluate model on test data
score = model.evaluate(X_test, Y_test, verbose=0)
print("score: ",score)

[[ -0.25]
 [  2.76]
 [ -0.98]
 [  1.72]
 [ -0.02]
 [  0.1 ]
 [ -0.42]
 [  7.38]
 [  0.27]
 [  0.86]
 [ -0.6 ]
 [ 13.13]
 [  0.77]
 [ -0.08]
 [  4.08]
 [  0.81]
 [ -0.36]
 [  0.98]
 [  0.42]
 [  0.35]]
[ 500.]
[-400.]
[[ -0.25]
 [  2.76]
 [ -0.98]
 [  1.72]
 [ -0.02]
 [  0.1 ]
 [ -0.42]
 [  7.38]
 [  0.27]
 [  0.86]
 [ -0.6 ]
 [ 13.13]
 [  0.77]
 [ -0.08]
 [  4.08]
 [  0.81]
 [ -0.36]
 [  0.98]
 [  0.42]
 [  0.35]]
[ 400.]
[-300.]
compile
summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 600)               464400    
_________________________________________________________________
dense_28 (Dense)             (None, 400)               240400    
_________________________________________________________________
dense_29 (Dense)             (None, 200)               80200     
_________________________________________________________________
dense_30 (Dense)      

In [49]:
model.predict(np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
 0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,1 ,0 ,0 ,0 ,0,
 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

ValueError: Error when checking : expected dense_27_input to have shape (None, 773) but got array with shape (773, 1)